# Hybrid Recommendation System

### Zumarsiyah Mahsyari

##### Astra Data Scientist Bootcamp

Hybrid recommendation system adalah metode dalam recommendation system yang menggabungkan algoritma Collaborative Filtering dengan Content-Based. Dalam Practice Case ini, kita akan mencoba mencari tahu genre input favorit dari film dan peringkat yang diberikan. <br>

Data yang digunakan pada Practice Case kali ini adalah dataset movies dan ratings. Data movies berisi daftar kumpulan film yang terdiri dari 3 feature, yaitu movieId, title (judul film), dan genres film. Sedangkan data ratings berisi kumpulan rating yang diberikan oleh user untuk beberapa film. Data ini terdiri dari userId, movieId, rating film, dan timestamp.
Algoritma ini mencoba untuk mencari tahu apa aspek favorit pengguna dari suatu item, dan kemudian merekomendasikan item yang menyajikan aspek-aspek tersebut. Dalam Practice Case ini, kita akan mencoba mencari tahu genre input favorit dari film dan peringkat yang diberikan.

In [ ]:
import pandas as pd
import numpy as np

# Load & Read Data

In [6]:
df = pd.read_csv('C:/Users/zumars064124/Documents/ADSB 2/Tugas MBA/moviedataset/ratings.csv',usecols=[0,1,2])
df.head()

userId  movieId  rating
0       1      169     2.5
1       1     2471     3.0
2       1    48516     5.0
3       2     2571     3.5
4       2   109487     4.0

In [7]:
df.shape

(22884377, 3)

Karena keterbatasan kemampuan untuk menganalisis seluruh data, maka kita ambil sampel dari data tersebut sebanyak 500000.

In [8]:
df = df.sample(500000)

In [9]:
raters=df.groupby(['movieId'])['rating'].count()
raters.head()

movieId
1    1319
2     529
3     326
4      62
5     305
Name: rating, dtype: int64

Kita akan mencari nilai korelasi untuk setiap film dengan setiap film lainnya menggunakan rating. Ini bagian dari algoritma Collaborative Filtering.

In [10]:
data=df.merge(raters,on='movieId')
data['count']=data['rating_y']
data['rating']= data['rating_x']
data.drop(['rating_x','rating_y'],axis=1,inplace=True)
data.head()

userId  movieId  count  rating
0  177917    38038    105     4.5
1   79390    38038    105     4.5
2   30237    38038    105     4.0
3  109211    38038    105     4.5
4  172635    38038    105     3.5

In [11]:
data.shape

(500000, 4)

In [12]:
selfjoined = data.merge(data,on='userId')
selfjoined.head()

userId  movieId_x  count_x  rating_x  movieId_y  count_y  rating_y
0  177917      38038      105       4.5      38038      105       4.5
1  177917      38038      105       4.5       1719      100       4.0
2  177917      38038      105       4.5      66304        5       1.0
3  177917       1719      100       4.0      38038      105       4.5
4  177917       1719      100       4.0       1719      100       4.0

In [13]:
selfjoined = selfjoined[selfjoined['movieId_x']<selfjoined['movieId_y']]

In [14]:
selfjoined['r1r2'] = selfjoined['rating_x']*selfjoined['rating_y']
selfjoined['r1square'] = np.square(selfjoined['rating_x'])
selfjoined['r2square'] = np.square(selfjoined['rating_y'])
selfjoined.head()

userId  movieId_x  count_x  rating_x  movieId_y  count_y  rating_y  r1r2  \
2   177917      38038      105       4.5      66304        5       1.0   4.5   
3   177917       1719      100       4.0      38038      105       4.5  18.0   
5   177917       1719      100       4.0      66304        5       1.0   4.0   
18   79390      38038      105       4.5      74458      215       4.0  18.0   
22   79390      38038      105       4.5      54999       32       4.0  18.0   

    r1square  r2square  
2      20.25      1.00  
3      16.00     20.25  
5      16.00      1.00  
18     20.25     16.00  
22     20.25     16.00

In [15]:
aggdata=selfjoined.groupby(['movieId_x','movieId_y'])['rating_x','rating_y','r1r2','r1square','r2square','count_x','count_y'].sum()
aggdata.head()

rating_x  rating_y   r1r2  r1square  r2square  count_x  \
movieId_x movieId_y                                                           
1         2              24.5      18.0  67.25     92.25     51.50     9233   
          3              16.0      16.5  52.00     54.50     55.25     6595   
          4               5.0       2.0  10.00     25.00      4.00     1319   
          5               7.0       5.0  17.00     25.00     13.00     2638   
          6              13.0      13.0  41.50     43.00     43.50     5276   

                     count_y  
movieId_x movieId_y           
1         2             3703  
          3             1630  
          4               62  
          5              610  
          6             2256

In [16]:
n=aggdata.shape[0]
n

2118666

In [17]:
aggdata['corelation'] = (n*aggdata['r1r2'] - aggdata['rating_x']*aggdata['rating_y']) / \
np.sqrt((n * aggdata['r1square']  - np.square(aggdata['rating_x']))*(n * aggdata['r2square']  - np.square(aggdata['rating_y'])))
aggdata['corelation'] = aggdata['corelation']*n/(n+50)

In [18]:
aggdata.reset_index(inplace=True)
aggdata.head()

movieId_x  movieId_y  rating_x  rating_y   r1r2  r1square  r2square  \
0          1          2      24.5      18.0  67.25     92.25     51.50   
1          1          3      16.0      16.5  52.00     54.50     55.25   
2          1          4       5.0       2.0  10.00     25.00      4.00   
3          1          5       7.0       5.0  17.00     25.00     13.00   
4          1          6      13.0      13.0  41.50     43.00     43.50   

   count_x  count_y  corelation  
0     9233     3703    0.975653  
1     6595     1630    0.947608  
2     1319       62    0.999976  
3     2638      610    0.942968  
4     5276     2256    0.959531

Setelah mendapatkan nilai korelasi dari setiap film dengan film lainnya, mari kita tambahkan judul film agar mudah membaca informasi

In [19]:
movies = pd.read_csv('C:/Users/zumars064124/Documents/ADSB 2/Tugas MBA/moviedataset/movies.csv',usecols=[0,1])
movies.head()

movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)

In [20]:
aggdata1=aggdata.merge(movies,left_on='movieId_x',right_on='movieId')
aggdata2=aggdata1.merge(movies,left_on='movieId_y',right_on='movieId')
aggdata2.head()         

movieId_x  movieId_y  rating_x  rating_y   r1r2  r1square  r2square  \
0          1          2      24.5      18.0  67.25     92.25     51.50   
1          1          3      16.0      16.5  52.00     54.50     55.25   
2          2          3       4.0       4.0  16.00     16.00     16.00   
3          1          4       5.0       2.0  10.00     25.00      4.00   
4          2          4       4.0       3.0  12.00     16.00      9.00   

   count_x  count_y  corelation  movieId_x           title_x  movieId_y  \
0     9233     3703    0.975653          1  Toy Story (1995)          2   
1     6595     1630    0.947608          1  Toy Story (1995)          3   
2      529      326    0.999976          2    Jumanji (1995)          3   
3     1319       62    0.999976          1  Toy Story (1995)          4   
4      529       62    0.999976          2    Jumanji (1995)          4   

                    title_y  
0            Jumanji (1995)  
1   Grumpier Old Men (1995)  
2   Grumpier Old Men (1995)  
3  Waiting to Exhale (1995)  
4  Waiting to Exhale (1995)

In [21]:
aggdata2.shape

(2118666, 14)

Kita akan melanjutkan dengan algoritma Content-Based di mana kita akan mencari rekomendasi film berdasarkan genre.

In [22]:
movies_df = pd.read_csv('C:/Users/zumars064124/Documents/ADSB 2/Tugas MBA/moviedataset/movies.csv')
movies_df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [23]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

In [24]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                     [Adventure, Children, Fantasy]  1995  
2                                  [Comedy, Romance]  1995  
3                           [Comedy, Drama, Romance]  1995  
4                                           [Comedy]  1995

In [25]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  Adventure  \
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                     [Adventure, Children, Fantasy]  1995        1.0   
2                                  [Comedy, Romance]  1995        0.0   
3                           [Comedy, Drama, Romance]  1995        0.0   
4                                           [Comedy]  1995        0.0   

   Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0        1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1        0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
2        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
3        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
4        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

   Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
2     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
3     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
4     0.0   0.0          0.0  0.0      0.0      0.0        0.0   

   (no genres listed)  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  

[5 rows x 24 columns]

In [26]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

rating                title
0     5.0  Breakfast Club, The
1     3.5            Toy Story
2     2.0              Jumanji
3     5.0         Pulp Fiction
4     4.5                Akira

In [27]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0

In [28]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  Adventure  \
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                        [Adventure, Children, Fantasy]  1995        1.0   
293                    [Comedy, Crime, Drama, Thriller]  1994        0.0   
1246             [Action, Adventure, Animation, Sci-Fi]  1988        1.0   
1885                                    [Comedy, Drama]  1985        0.0   

      Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0           1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1           0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
293         0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   
1246        1.0       0.0     0.0      0.0      0.0  ...     0.0      0.0   
1885        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

      Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0        0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1        0.0   0.0          0.0  0.0      0.0      0.0        0.0   
293      0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1246     1.0   0.0          0.0  0.0      0.0      0.0        0.0   
1885     0.0   0.0          0.0  0.0      0.0      0.0        0.0   

      (no genres listed)  
0                    0.0  
1                    0.0  
293                  0.0  
1246                 0.0  
1885                 0.0  

[5 rows x 24 columns]

In [29]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  Action  \
0        1.0        1.0       1.0     1.0      1.0      0.0    0.0     0.0   
1        1.0        0.0       1.0     0.0      1.0      0.0    0.0     0.0   
2        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   
3        1.0        1.0       0.0     0.0      0.0      0.0    0.0     1.0   
4        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   

   Crime  Thriller  Horror  Mystery  Sci-Fi  IMAX  Documentary  War  Musical  \
0    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
1    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
2    1.0       1.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
3    0.0       0.0     0.0      0.0     1.0   0.0          0.0  0.0      0.0   
4    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   

   Western  Film-Noir  (no genres listed)  
0      0.0        0.0                 0.0  
1      0.0        0.0                 0.0  
2      0.0        0.0                 0.0  
3      0.0        0.0                 0.0  
4      0.0        0.0                 0.0

In [30]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [31]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  \
movieId                                                                    
1              1.0        1.0       1.0     1.0      1.0      0.0    0.0   
2              1.0        0.0       1.0     0.0      1.0      0.0    0.0   
3              0.0        0.0       0.0     1.0      0.0      1.0    0.0   
4              0.0        0.0       0.0     1.0      0.0      1.0    1.0   
5              0.0        0.0       0.0     1.0      0.0      0.0    0.0   

         Action  Crime  Thriller  Horror  Mystery  Sci-Fi  IMAX  Documentary  \
movieId                                                                        
1           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
2           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
3           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
4           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
5           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   

         War  Musical  Western  Film-Noir  (no genres listed)  
movieId                                                        
1        0.0      0.0      0.0        0.0                 0.0  
2        0.0      0.0      0.0        0.0                 0.0  
3        0.0      0.0      0.0        0.0                 0.0  
4        0.0      0.0      0.0        0.0                 0.0  
5        0.0      0.0      0.0        0.0                 0.0

In [32]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [33]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

In [41]:
#The recommendation table for content-based
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(5).keys())]

movieId                                              title  \
4923      5018                                           Motorama   
6793      6902                                      Interstate 60   
8605     26093         Wonderful World of the Brothers Grimm, The   
9296     27344  Revolutionary Girl Utena: Adolescence of Utena...   
33509   148775                Wizards of Waverly Place: The Movie   

                                                  genres  year  
4923   [Adventure, Comedy, Crime, Drama, Fantasy, Mys...  1991  
6793   [Adventure, Comedy, Drama, Fantasy, Mystery, S...  2002  
8605   [Adventure, Animation, Children, Comedy, Drama...  1962  
9296   [Action, Adventure, Animation, Comedy, Drama, ...  1999  
33509  [Adventure, Children, Comedy, Drama, Fantasy, ...  2009

Tabel di atas merupakan hasil rekomendasi dari content-based. Salah satu film yang direkomendasikan adalah Interstate 60 (2002). Mari kita gunakan korelasi yang didapatkan dari Collaborative Filtering tadi berdasarkan film Interstate 60 (2002) untuk mendapatkan rekomendasi film.

In [37]:
def recommendation(title):
    recommended_movies = []
    data =aggdata2[aggdata2['title_x']==title]
    data = data.sort_values(by='corelation',ascending=False)
    return data


collabaritive=recommendation('Interstate 60 (2002)')[['title_y','corelation']]
collabaritive['title'] = collabaritive['title_y'].str.split(' \(').str.get(0)
collabaritive.drop('title_y',axis=1,inplace=True)
collabaritive.head()

corelation                                       title
705624     0.999976                               Love Actually
1530727    0.999976                             In Good Company
1712643    0.999976                                  Dark Skies
1709080    0.999976                          Last Chance Harvey
1698032    0.999976  Star Wars: Episode VII - The Force Awakens

5 film di atas adalah film yang direkomendasikan menggunakan Hybrid Recommendation System.